In [131]:
import numpy as np
import os

In [132]:
### CLASSES ###
class Cell:
    def __init__(self, input_val):
        self.value = 0
        if(input_val != '*'):
            self.value = int(input_val)


class Board:
    def __init__(self, file_name):
        # Get Boards folder
        current_directory = os.getcwd() #Current working directory
        parent_directory = os.path.dirname(current_directory)
        new_folder_path = os.path.join(parent_directory, 'Unsolved-Boards')
        text_file_path = os.path.join(new_folder_path, file_name)

        # Check if the folder exists, if not, create it
        if not os.path.exists(text_file_path):
            print("ERROR - the path " + text_file_path + " does not exist")
            return

        
        # Read File
        fileReader = open(text_file_path, "r")    
        file_as_text = fileReader.read()
        text_to_array = file_as_text.split('\n')
        text_to_array = np.array([[tile for tile in row] for row in text_to_array if len(row) > 0])
        #print(text_to_array)
        
        
        # Turn File into Board object
        self.board = []
        
        for row in text_to_array:
            row_list = []
            for element in row:
                row_list.append(Cell(element))
                
            self.board.append(row_list)
        
    def print_board(self):
        for row in self.board:
            string = ""
            
            for cell in row:
                if cell.value == 0:
                    string += "*" + " "
                else:
                    string += str(cell.value) + " "
                
            print(string)

        


In [133]:
### BACK TRACKING ALGORTHM ###

#Creating the Constraint Satisfaction Problem variable:
sudoku_csp = {
    #The variable item contains a list of the cells of the Sudoku Board
    'variables': [(i, j) for i in range(9) for j in range(9)],
    
    #The domain item contains a dictionary of each variable containing a list of numbers from 1 to 9
    'domains': {variable: list(range(1, 10)) for variable in [(i, j) for i in range(9) for j in range(9)]},
    
    #The constraint item contains a dictionary of each constraint type
    #Each constraint hold a lamba function for checking if a given value violates the contraint or not
    'constraints': {
        'row': lambda board, row, col, num: num not in board[row],
        'column': lambda board, row, col, num: num not in [board[i][col] for i in range(9)],
        'subgrid': lambda board, row, col, num: all(num != board[i][j] for i in range(3*(row//3), 3*(row//3) + 3) for j in range(3*(col//3), 3*(col//3) + 3))
    }
}

In [134]:
#The class for solving the Puzzle
def solve_sudoku_csp(board, csp):
    
    #Creating variables holding the different components of the CSP variable
    variables = csp['variables']
    domains = csp['domains']
    constraints = csp['constraints']
    
    #Creating a dictionary to store the assigned cells
    assigned_cells = {}  
    
    #the backtracking function
    def backtrack(board):
        
        #Calls the function to find an unassigned cell
        empty_cell = find_empty_cell(board)
        
        #if the empty cell returns None, the board is solved
        if not empty_cell:
            return True
        
        #set the row and column of the empty cell
        row, col = empty_cell
        
        #go through each value in the domain of the empty cell to find the correct value
        for num in domains[empty_cell]:
            
            #if the current value does not violate any constraint, assign the value to the empty cell
            if is_valid(board, row, col, num, constraints):
                board[row][col] = num
                
                #Record the assigned cell
                assigned_cells[(row, col)] = num
                
                #call the backtracking function for another empty cell
                #if there are no more empty cells, return true
                if backtrack(board):
                    return True
                #Backtrack and remove the cell
                board[row][col] = 0  
                del assigned_cells[(row, col)] 
                
        #No solution found
        return False  
    
    #the function to find an empty cell
    def find_empty_cell(board):
        
        #Go through each row and column to find a cell that contains 0
        for i in range(9):
            for j in range(9):
                if board[i][j] == 0:
                    return (i, j)
        return None
    
    #the function to check the board's consistency with the constraints
    def is_valid(board, row, col, num, constraints):
        
        #looping to go through each constraint and using the function inside to test for consistency
        for constraint_type, constraint_func in constraints.items():
            
            #if the function does not return true, then it is not consistent
            if not constraint_func(board, row, col, num):
                return False
        return True
    
    #if the backtracking function returns true, return the board
    if backtrack(board):
        return True, board, assigned_cells  # Return assigned cells along with the solution
    else:
        return False, None, None



In [135]:
game1 = Board("Puzzle_1.txt")
game1.print_board()

8 2 1 * * * * * 7 
* * * 8 * * * 6 * 
* 6 * 9 3 * * * 5 
* * 8 2 * 1 6 * * 
* * * 7 * * 2 8 4 
2 4 * 6 * 3 7 * * 
6 * 5 * * * 1 * 3 
* 7 * * 5 * * * * 
9 1 2 * * * * * 6 


In [136]:
#function for copying the board into a list of lists of ints
def CopyBoard(board):
    newBoardList = []
    for row in board:
    
        row_list = [] 
        for cell in row:
            row_list.append(int(cell.value))
    
        newBoardList.append(row_list)
        
    return newBoardList
    
    
    

In [137]:
#Creating a new board to get the integer values of each row
unsolvedBoard = CopyBoard(game1.board)

unsolvedBoard

[[8, 2, 1, 0, 0, 0, 0, 0, 7],
 [0, 0, 0, 8, 0, 0, 0, 6, 0],
 [0, 6, 0, 9, 3, 0, 0, 0, 5],
 [0, 0, 8, 2, 0, 1, 6, 0, 0],
 [0, 0, 0, 7, 0, 0, 2, 8, 4],
 [2, 4, 0, 6, 0, 3, 7, 0, 0],
 [6, 0, 5, 0, 0, 0, 1, 0, 3],
 [0, 7, 0, 0, 5, 0, 0, 0, 0],
 [9, 1, 2, 0, 0, 0, 0, 0, 6]]

In [138]:
solved, solution, assignments = solve_sudoku_csp(unsolvedBoard, sudoku_csp)
if solved:
    print("Sudoku Solved Successfully!")
    for row in solution:
        print(row)
else:
    print("No solution exists!")

Sudoku Solved Successfully!
[8, 2, 1, 5, 6, 4, 3, 9, 7]
[5, 9, 3, 8, 1, 7, 4, 6, 2]
[4, 6, 7, 9, 3, 2, 8, 1, 5]
[7, 5, 8, 2, 4, 1, 6, 3, 9]
[1, 3, 6, 7, 9, 5, 2, 8, 4]
[2, 4, 9, 6, 8, 3, 7, 5, 1]
[6, 8, 5, 4, 2, 9, 1, 7, 3]
[3, 7, 4, 1, 5, 6, 9, 2, 8]
[9, 1, 2, 3, 7, 8, 5, 4, 6]


In [139]:
game2 = Board("Puzzle_2.txt")
game2.print_board()

5 3 * * 7 * * * * 
6 * * 1 9 5 * * * 
* 9 8 * * * * 6 * 
8 * * * 6 * * * 3 
4 * * 8 * 3 * * 1 
7 * * * 2 * * * 6 
* 6 * * * * 2 8 * 
* * * 4 1 9 * * 5 
* * * * 8 * * 7 9 


In [140]:
unsolvedBoard = CopyBoard(game2.board)

unsolvedBoard

[[5, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [141]:
solved, solution, assignments = solve_sudoku_csp(unsolvedBoard, sudoku_csp)
if solved:
    print("Sudoku Solved Successfully!")
    for row in solution:
        print(row)
else:
    print("No solution exists!")

Sudoku Solved Successfully!
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


In [142]:
### SOLVING STRATEGIES ###



In [143]:
### FRONT END CONNECTION ###
"""from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/calculate', methods=['GET'])
def calculate():
    # Perform backend calculations
    result = perform_calculations()

    # Return the result as JSON
    return jsonify({'result': result})

def perform_calculations():
    # Your calculation logic here
    return calculated_result

if __name__ == '__main__':
    app.run(debug=True)"""
print()

In [144]:
### COMBINED SOLVING AND MAIN FUNCTION ###